In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import train_test_split
from sklearn import tree
import pandas_redshift as pr
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
import datetime, pytz, sys
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from skopt import BayesSearchCV
from sklearn.model_selection import StratifiedKFold
from math import sqrt, log
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

/home/rabkol/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def model_accuracy(trained_model, features, targets):
    accuracy_score = trained_model.score(features, targets)
    return accuracy_score

In [3]:
df_train = pd.read_csv('/local/home/rabkol/Desktop/01_Projects/Leaderboard/MLE_Tiny_Training_Trial/TTT_train.csv')
#df_test = pd.read_csv('/local/home/rabkol/Desktop/01_Projects/Leaderboard/MLE_Tiny_Training_Trial/TTT_test_features.csv')

In [4]:
training_features = list(df_train.columns.values)

In [5]:
training_features = list(df_train.columns.values)
training_features.remove('label')
target = 'label'

In [6]:
#Stratified Sampling
for i in range(10):
    df_train_samp = df_train[df_train.label == i]
    train_x_samp, test_x_samp, train_y_samp, test_y_samp = train_test_split(df_train_samp[training_features], df_train_samp[target], train_size=0.8)
    if i == 0:
        train_x = train_x_samp
        test_x = test_x_samp
        train_y = train_y_samp
        test_y = test_y_samp
    else:
        train_x = pd.concat([train_x, train_x_samp], axis=0)
        test_x = pd.concat([test_x, test_x_samp], axis=0)
        train_y = pd.concat([train_y, train_y_samp], axis=0)
        test_y = pd.concat([test_y, test_y_samp], axis=0)

In [7]:
print('Train:\n',train_y.value_counts()*100/train_y.count())
print('Test:\n',test_y.value_counts()*100/test_y.count())

Train:
 9    29.292929
0    25.353535
8    13.535354
7     6.464646
6     5.555556
5     5.252525
4     5.252525
3     3.636364
2     2.828283
1     2.828283
Name: label, dtype: float64
Test:
 9    28.740157
0    24.803150
8    13.385827
7     6.692913
6     5.511811
5     5.511811
4     5.118110
3     3.937008
2     3.149606
1     3.149606
Name: label, dtype: float64


In [10]:
#Get Accuracy of Base Model RF
RF_base = None
#Auto Growing Depth of Tree
test_accuracy_prior = 0
for depth in range(2,50,10):
    clf = RandomForestClassifier(n_estimators = 1, max_depth=depth, min_samples_split = 3, min_samples_leaf = 1,n_jobs = -1)
    clf.fit(train_x, train_y)
    test_accuracy = model_accuracy(clf, test_x, test_y)
    train_accuracy = model_accuracy(clf, train_x, train_y)
    datetime_print = datetime.datetime.now(pytz.timezone('US/Pacific')).strftime('%Y-%m-%d %H:%M:%S %Z%z')
    print(datetime_print,'\t:\tStage 1, Depth ',depth,'\t:\t',train_accuracy,',',test_accuracy)
    if (test_accuracy - test_accuracy_prior) <= 0.0010:#0.10% improvement in test accuracy between subsequent increase in tree sizes
        #print('Depth Chosen for RF classifier\t:\t\t',depth,'\n')
        break
    test_accuracy_prior = test_accuracy
    sys.stdout.flush()

#Auto Growing Number of Trees
test_accuracy_prior = 0
for trees in range(5,500,100):
    clf = RandomForestClassifier(n_estimators = trees, max_depth=depth, min_samples_split = 3, min_samples_leaf = 1,n_jobs = -1)
    clf.fit(train_x, train_y)
    test_accuracy = model_accuracy(clf, test_x, test_y)
    train_accuracy = model_accuracy(clf, train_x, train_y)
    datetime_print = datetime.datetime.now(pytz.timezone('US/Pacific')).strftime('%Y-%m-%d %H:%M:%S %Z%z')
    print(datetime_print,'\t:\tStage 1, Depth ',depth,'\t:\t',train_accuracy,',',test_accuracy)
    if (test_accuracy - test_accuracy_prior) <= 0.0010:#0.10% improvement in test accuracy between subsequent increase in tree sizes
        #print('Num Trees Chosen for RF classifier\t:\t\t',trees,'\n')
        RF_base = clf
        break
    test_accuracy_prior = test_accuracy
    sys.stdout.flush()

2019-04-15 14:48:18 PDT-0700 	:	Stage 1, Depth  2 	:	 0.35555555555555557 , 0.36220472440944884
2019-04-15 14:48:18 PDT-0700 	:	Stage 1, Depth  12 	:	 0.509090909090909 , 0.4645669291338583
2019-04-15 14:48:19 PDT-0700 	:	Stage 1, Depth  22 	:	 0.6989898989898989 , 0.6377952755905512
2019-04-15 14:48:19 PDT-0700 	:	Stage 1, Depth  32 	:	 0.7636363636363637 , 0.6535433070866141
2019-04-15 14:48:19 PDT-0700 	:	Stage 1, Depth  42 	:	 0.8252525252525252 , 0.65748031496063
2019-04-15 14:48:19 PDT-0700 	:	Stage 1, Depth  42 	:	 0.9282828282828283 , 0.7440944881889764
2019-04-15 14:48:20 PDT-0700 	:	Stage 1, Depth  42 	:	 0.9717171717171718 , 0.7598425196850394
2019-04-15 14:48:20 PDT-0700 	:	Stage 1, Depth  42 	:	 0.9767676767676767 , 0.7677165354330708
2019-04-15 14:48:22 PDT-0700 	:	Stage 1, Depth  42 	:	 0.9767676767676767 , 0.7716535433070866
2019-04-15 14:48:23 PDT-0700 	:	Stage 1, Depth  42 	:	 0.9767676767676767 , 0.7637795275590551


In [14]:
# Introduce some randomness for better generalizability and low data size in training
# Add Random Noise (Augment existing training data)

def rand_noise_inserter(dataset,n):
    df_train_rand = dataset
    for i in range(n):
        df_randn = pd.DataFrame(np.random.uniform(0.05,0.15)*np.random.randn(dataset.shape[0], dataset.shape[1]-1) + 1, columns=list(dataset.columns.values)[:-1])
        #df_randn = pd.DataFrame(max_sd*np.random.randn(dataset.shape[0], dataset.shape[1]-1) + 1, columns=list(dataset.columns.values)[:-1])
        df_randn['label'] = 1
        df_randn.index = dataset.index
        df_train_rand_temp = dataset * df_randn
        df_train_rand = pd.concat([df_train_rand, df_train_rand_temp])
    df_train_rand2 = df_train_rand
    df_train_rand = df_train_rand2.clip(0,1)
    df_train_rand['label'] = df_train_rand2['label']
    df_train_rand.reset_index()
    return df_train_rand

train_xy = pd.concat([train_x, train_y], axis = 1)
train_xy_rand = rand_noise_inserter(train_xy,100)
print(train_xy.shape,',',train_xy_rand.shape)

(990, 1257) , (99990, 1257)


In [15]:
train_xy_rand.tail()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f1247,f1248,f1249,f1250,f1251,f1252,f1253,f1254,f1255,label
1202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9
846,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9
906,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9
868,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9
985,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9


In [16]:
#Get Accuracy of RF w/ data augmentation
RF_Noise = None
#Auto Growing Depth of Tree
test_accuracy_prior = 0
for depth in range(2,50,10):
    clf = RandomForestClassifier(n_estimators = 1, max_depth=depth, min_samples_split = 2, min_samples_leaf = 1,n_jobs = -1)
    clf.fit(train_xy_rand[training_features], train_xy_rand[target])
    test_accuracy = model_accuracy(clf, test_x, test_y)
    train_accuracy = model_accuracy(clf, train_xy_rand[training_features], train_xy_rand[target])
    datetime_print = datetime.datetime.now(pytz.timezone('US/Pacific')).strftime('%Y-%m-%d %H:%M:%S %Z%z')
    print(datetime_print,'\t:\tStage 1, Depth ',depth,'\t:\t',train_accuracy,',',test_accuracy)
    if (test_accuracy - test_accuracy_prior) <= 0.0010:#0.10% improvement in test accuracy between subsequent increase in tree sizes
        #print('Depth Chosen for RF classifier\t:\t\t',depth,'\n')
        break
    test_accuracy_prior = test_accuracy
    sys.stdout.flush()

#Auto Growing Number of Trees
test_accuracy_prior = 0
for trees in range(5,1000,400):
    clf = RandomForestClassifier(n_estimators = trees, max_depth=depth, min_samples_split = 2, min_samples_leaf = 1,n_jobs = -1)
    clf.fit(train_xy_rand[training_features], train_xy_rand[target])
    test_accuracy = model_accuracy(clf, test_x, test_y)
    train_accuracy = model_accuracy(clf, train_xy_rand[training_features], train_xy_rand[target])
    datetime_print = datetime.datetime.now(pytz.timezone('US/Pacific')).strftime('%Y-%m-%d %H:%M:%S %Z%z')
    print(datetime_print,'\t:\tStage 2, Width ',trees,'\t:\t',train_accuracy,',',test_accuracy)
    if (test_accuracy - test_accuracy_prior) <= 0.0010:#0.10% improvement in test accuracy between subsequent increase in tree sizes
        #print('Num Trees Chosen for RF classifier\t:\t\t',trees,'\n')
        RF_Noise = clf
        break
    test_accuracy_prior = test_accuracy
    sys.stdout.flush()

2019-04-15 14:52:53 PDT-0700 	:	Stage 1, Depth  2 	:	 0.33434343434343433 , 0.3110236220472441
2019-04-15 14:52:56 PDT-0700 	:	Stage 1, Depth  12 	:	 0.5158015801580158 , 0.4251968503937008
2019-04-15 14:52:58 PDT-0700 	:	Stage 1, Depth  22 	:	 0.6542454245424543 , 0.531496062992126
2019-04-15 14:53:01 PDT-0700 	:	Stage 1, Depth  32 	:	 0.8153915391539154 , 0.5866141732283464
2019-04-15 14:53:04 PDT-0700 	:	Stage 1, Depth  42 	:	 0.8850785078507851 , 0.65748031496063
2019-04-15 14:53:07 PDT-0700 	:	Stage 2, Width  5 	:	 0.9616861686168617 , 0.7440944881889764
2019-04-15 14:53:27 PDT-0700 	:	Stage 2, Width  405 	:	 0.9892689268926893 , 0.7913385826771654
2019-04-15 14:54:02 PDT-0700 	:	Stage 2, Width  805 	:	 0.98994899489949 , 0.7913385826771654


In [21]:
#train_xy = pd.concat([train_x, train_y], axis = 1)
df_train_rand = rand_noise_inserter(df_train,100)
print(df_train.shape,',',df_train_rand.shape)

(1244, 1257) , (125644, 1257)


In [22]:
#Train on entire dataset + data augmentation. Test accuracy scores are meaningless here. 
RF_Noise = None
#Auto Growing Depth of Tree
test_accuracy_prior = 0
for depth in range(2,50,10):
    clf = RandomForestClassifier(n_estimators = 1, max_depth=depth, min_samples_split = 2, min_samples_leaf = 1,n_jobs = -1)
    clf.fit(df_train_rand[training_features], df_train_rand[target])
    test_accuracy = model_accuracy(clf, test_x, test_y)
    train_accuracy = model_accuracy(clf, df_train_rand[training_features], df_train_rand[target])
    datetime_print = datetime.datetime.now(pytz.timezone('US/Pacific')).strftime('%Y-%m-%d %H:%M:%S %Z%z')
    print(datetime_print,'\t:\tStage 1, Depth ',depth,'\t:\t',train_accuracy,',',test_accuracy)
    if (test_accuracy - test_accuracy_prior) <= 0.0010:#0.10% improvement in test accuracy between subsequent increase in tree sizes
        #print('Depth Chosen for RF classifier\t:\t\t',depth,'\n')
        break
    test_accuracy_prior = test_accuracy
    sys.stdout.flush()

#Auto Growing Number of Trees
test_accuracy_prior = 0
for trees in range(5,1000,400):
    clf = RandomForestClassifier(n_estimators = trees, max_depth=depth, min_samples_split = 2, min_samples_leaf = 1,n_jobs = -1)
    clf.fit(df_train_rand[training_features], df_train_rand[target])
    test_accuracy = model_accuracy(clf, test_x, test_y)
    train_accuracy = model_accuracy(clf, df_train_rand[training_features], df_train_rand[target])
    datetime_print = datetime.datetime.now(pytz.timezone('US/Pacific')).strftime('%Y-%m-%d %H:%M:%S %Z%z')
    print(datetime_print,'\t:\tStage 2, Width ',trees,'\t:\t',train_accuracy,',',test_accuracy)
    if (test_accuracy - test_accuracy_prior) <= 0.0010:#0.10% improvement in test accuracy between subsequent increase in tree sizes
        #print('Num Trees Chosen for RF classifier\t:\t\t',trees,'\n')
        RF_Noise = clf
        break
    test_accuracy_prior = test_accuracy
    sys.stdout.flush()

2019-04-15 15:09:37 PDT-0700 	:	Stage 1, Depth  2 	:	 0.32556270096463025 , 0.3228346456692913
2019-04-15 15:09:40 PDT-0700 	:	Stage 1, Depth  12 	:	 0.49220018464869 , 0.5039370078740157
2019-04-15 15:09:43 PDT-0700 	:	Stage 1, Depth  22 	:	 0.574615580529114 , 0.5905511811023622
2019-04-15 15:09:46 PDT-0700 	:	Stage 1, Depth  32 	:	 0.7910604565279679 , 0.8070866141732284
2019-04-15 15:09:50 PDT-0700 	:	Stage 1, Depth  42 	:	 0.8814268886695743 , 0.8858267716535433
2019-04-15 15:09:54 PDT-0700 	:	Stage 2, Width  5 	:	 0.9651714367578237 , 0.9606299212598425
2019-04-15 15:10:21 PDT-0700 	:	Stage 2, Width  405 	:	 0.9848062780554583 , 0.9803149606299213
2019-04-15 15:11:14 PDT-0700 	:	Stage 2, Width  805 	:	 0.985037088917895 , 0.9803149606299213


In [26]:
clf = RandomForestClassifier(n_estimators = 2000, max_depth=42, min_samples_split = 2, min_samples_leaf = 1,n_jobs = -1)
clf.fit(df_train_rand[training_features], df_train_rand[target])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=42, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=2000, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [25]:
df_test = pd.read_csv('/local/home/rabkol/Desktop/01_Projects/Leaderboard/MLE_Tiny_Training_Trial/TTT_test_features.csv')
ypred = pd.DataFrame(clf.predict(df_test[training_features]))
ypred.index = df_test.ID
ypred.to_csv('/local/home/rabkol/Desktop/01_Projects/Leaderboard/MLE_Tiny_Training_Trial/20190331_RF_BP_Train_All_2k_rand.csv', header=True, sep = ',')